# Modeling Notebook

End-to-end notebook: feature engineering, scaling, training (RandomForest + XGBoost), evaluation, and model export.

**Paths and outputs:**
- Input: `cleaned_engineered_data.csv` in the data folder
- Outputs: `walmart_sales_predictions.csv`, `rf_model.pkl`, `xgb_model.pkl`, `scaler.pkl`

Run cells top → bottom.

In [13]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBRegressor


print("="*60)
print("WALMART SALES FORECASTING - MODEL TRAINING")
print("="*60)

WALMART SALES FORECASTING - MODEL TRAINING


In [6]:
# STEP 1: LOAD DATA
# ============================================================
print("\n[STEP 1] Loading data...")

try:
    df = pd.read_csv("data/cleaned_engineered_data_cleaned.csv")
    print(f"✓ Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f"✓ Columns: {list(df.columns)}")
except FileNotFoundError:
    print("❌ ERROR: File 'data/cleaned_engineered_data_cleaned.csv' not found!")
    print("Please ensure the data file exists in the 'data' folder.")
    exit(1)


[STEP 1] Loading data...
✓ Data loaded: 10000 rows, 24 columns
✓ Columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size', 'IsHoliday', 'Month', 'Week', 'Year', 'Day', 'DayOfWeek', 'Weekly_Sales_Rolling4', 'Sales_vs_StoreMean', 'Type_B', 'Type_C']


In [7]:
# STEP 2: FEATURE ENGINEERING
# ============================================================
print("\n[STEP 2] Feature Engineering...")

# Date features (if Date column exists)
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Week'] = df['Date'].dt.isocalendar().week
    df.drop(columns=['Date'], inplace=True)
    print("✓ Date features created: Year, Month, Day, DayOfWeek, Week")
else:
    print("⚠ No 'Date' column found - skipping date feature creation")

# Fill missing values in numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
missing_count = 0
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())
        missing_count += 1

if missing_count > 0:
    print(f"✓ Missing values filled in {missing_count} numeric columns")
else:
    print("✓ No missing values found")

# Check for categorical columns that need encoding
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"✓ Categorical columns found: {categorical_cols if categorical_cols else 'None'}")

# IMPORTANT: We keep Store and Dept as NUMERIC, not one-hot encode them
# This is crucial for the UI to work properly
if 'Store' in df.columns:
    df['Store'] = df['Store'].astype(int)
    print("✓ Store column converted to integer")

if 'Dept' in df.columns:
    df['Dept'] = df['Dept'].astype(int)
    print("✓ Dept column converted to integer")

# Handle other categorical columns (if any) - like Type
if 'Type' in df.columns and df['Type'].dtype == 'object':
    # Keep Type_B and Type_C as they are (already encoded)
    df.drop(columns=['Type'], inplace=True, errors='ignore')
    print("✓ Removed 'Type' column (Type_B and Type_C already encoded)")

print(f"✓ Final feature count: {df.shape[1]} columns")


[STEP 2] Feature Engineering...
✓ Date features created: Year, Month, Day, DayOfWeek, Week
✓ Missing values filled in 1 numeric columns
✓ Categorical columns found: None
✓ Store column converted to integer
✓ Dept column converted to integer
✓ Final feature count: 23 columns


In [8]:
# STEP 3: PREPARE FEATURES AND TARGET
# ============================================================
print("\n[STEP 3] Preparing features and target...")

target_col = 'Weekly_Sales'

if target_col not in df.columns:
    print(f"❌ ERROR: Target column '{target_col}' not found in dataset!")
    print(f"Available columns: {list(df.columns)}")
    exit(1)

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

print(f"✓ Features (X): {X.shape}")
print(f"✓ Target (y): {y.shape}")
print(f"\n✓ Feature columns ({len(X.columns)}):")
for i, col in enumerate(X.columns, 1):
    print(f"   {i}. {col}")



[STEP 3] Preparing features and target...
✓ Features (X): (10000, 22)
✓ Target (y): (10000,)

✓ Feature columns (22):
   1. Store
   2. Dept
   3. Temperature
   4. Fuel_Price
   5. MarkDown1
   6. MarkDown2
   7. MarkDown3
   8. MarkDown4
   9. MarkDown5
   10. CPI
   11. Unemployment
   12. Size
   13. IsHoliday
   14. Month
   15. Week
   16. Year
   17. Day
   18. DayOfWeek
   19. Weekly_Sales_Rolling4
   20. Sales_vs_StoreMean
   21. Type_B
   22. Type_C


In [10]:
# STEP 4: TRAIN-TEST SPLIT
# ============================================================
print("\n[STEP 4] Train-Test Split...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✓ Training set: {X_train.shape[0]} samples")
print(f"✓ Test set: {X_test.shape[0]} samples")


[STEP 4] Train-Test Split...
✓ Training set: 8000 samples
✓ Test set: 2000 samples


In [11]:
# STEP 5: TRAIN RANDOM FOREST MODEL
# ============================================================
print("\n[STEP 5] Training Random Forest Model...")
print("⏳ This may take a few minutes...")

rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_model.fit(X_train, y_train)
print("✓ Model training complete!")


[STEP 5] Training Random Forest Model...
⏳ This may take a few minutes...
✓ Model training complete!


In [14]:
# STEP 5b: TRAIN XGBOOST MODEL
# ============================================================
print("\n[STEP 5b] Training XGBoost Model...")
print("⏳ This may take a few minutes...")

xgb_model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

xgb_model.fit(X_train, y_train)
print("✓ XGBoost training complete!")



[STEP 5b] Training XGBoost Model...
⏳ This may take a few minutes...
✓ XGBoost training complete!


In [9]:
# STEP 6: EVALUATE MODEL
# ============================================================
print("\n[STEP 6] Model Evaluation...")

# Predictions
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# Metrics function
def evaluate_model(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{dataset_name} Performance:")
    print(f"  MAE:  ${mae:,.2f}")
    print(f"  RMSE: ${rmse:,.2f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  MAPE: {mape:.2f}%")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}

train_metrics = evaluate_model(y_train, y_train_pred, "TRAINING SET")
test_metrics = evaluate_model(y_test, y_test_pred, "TEST SET")

# Feature Importance
print("\n[Feature Importance] Top 20 Features:")
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(feature_importance.head(20).to_string(index=False))


[STEP 6] Model Evaluation...

TRAINING SET Performance:
  MAE:  $21.62
  RMSE: $215.23
  R²:   0.9999
  MAPE: inf%

TEST SET Performance:
  MAE:  $35.91
  RMSE: $276.21
  R²:   0.9998
  MAPE: inf%

[Feature Importance] Top 20 Features:
              Feature   Importance
   Sales_vs_StoreMean 9.996272e-01
                Store 1.143402e-04
                 Size 1.041665e-04
                 Dept 6.198808e-05
         Unemployment 5.267525e-05
Weekly_Sales_Rolling4 1.157001e-05
          Temperature 8.438486e-06
                  CPI 7.485981e-06
                  Day 4.088504e-06
                 Week 3.046947e-06
           Fuel_Price 2.169864e-06
                Month 1.340863e-06
            IsHoliday 8.287108e-07
                 Year 1.716667e-07
            MarkDown1 1.263021e-07
            MarkDown5 8.782040e-08
            MarkDown2 8.380953e-08
            MarkDown4 7.958106e-08
            MarkDown3 7.823684e-08
            DayOfWeek 0.000000e+00


In [16]:
# Function to evaluate model performance
def evaluate_model(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{dataset_name} Performance:")
    print(f"  MAE:  ${mae:,.2f}")
    print(f"  RMSE: ${rmse:,.2f}")
    print(f"  R²:   {r2:.4f}")
    print(f"  MAPE: {mape:.2f}%")
    
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}


In [17]:
# STEP 6b: Evaluate XGBoost Model
# ============================================================
print("\n[STEP 6b] Evaluating XGBoost Model...")

y_train_pred_xgb = xgb_model.predict(X_train)
y_test_pred_xgb = xgb_model.predict(X_test)

train_metrics_xgb = evaluate_model(y_train, y_train_pred_xgb, "TRAINING SET (XGB)")
test_metrics_xgb = evaluate_model(y_test, y_test_pred_xgb, "TEST SET (XGB)")



[STEP 6b] Evaluating XGBoost Model...

TRAINING SET (XGB) Performance:
  MAE:  $65.50
  RMSE: $96.24
  R²:   1.0000
  MAPE: inf%

TEST SET (XGB) Performance:
  MAE:  $156.99
  RMSE: $408.08
  R²:   0.9996
  MAPE: inf%


In [19]:
# ============================================================
# SAVE XGBOOST METRICS (REQUIRED FOR STREAMLIT APP)
# ============================================================

metrics_xgb_df = pd.DataFrame({
    "Metric": ["MAE", "RMSE", "R2", "MAPE"],
    "Train": [
        train_metrics_xgb["MAE"],
        train_metrics_xgb["RMSE"],
        train_metrics_xgb["R2"],
        train_metrics_xgb["MAPE"]
    ],
    "Test": [
        test_metrics_xgb["MAE"],
        test_metrics_xgb["RMSE"],
        test_metrics_xgb["R2"],
        test_metrics_xgb["MAPE"]
    ]
})

metrics_xgb_df.to_csv("model_metrics_xgb.csv", index=False)
print("✓ XGBoost metrics saved: model_metrics_xgb.csv")


✓ XGBoost metrics saved: model_metrics_xgb.csv


In [18]:
# STEP 7b: Save XGBoost Model
# ============================================================
joblib.dump(xgb_model, 'xgb_model.pkl')
print("✓ XGBoost model saved: xgb_model.pkl")


✓ XGBoost model saved: xgb_model.pkl


In [10]:
# STEP 7: SAVE MODEL AND ARTIFACTS
# ============================================================
print("\n[STEP 7] Saving model and artifacts...")

# Save the trained model
joblib.dump(rf_model, 'rf_model.pkl')
print("✓ Model saved: rf_model.pkl")

# Save feature names (column order is critical!)
feature_names = list(X.columns)
joblib.dump(feature_names, 'input_features.pkl')
print("✓ Feature names saved: input_features.pkl")

# Save feature importance
feature_importance.to_csv('feature_importance.csv', index=False)
print("✓ Feature importance saved: feature_importance.csv")

# Save model metrics
metrics_df = pd.DataFrame({
    'Metric': ['MAE', 'RMSE', 'R2', 'MAPE'],
    'Train': [train_metrics['MAE'], train_metrics['RMSE'], train_metrics['R2'], train_metrics['MAPE']],
    'Test': [test_metrics['MAE'], test_metrics['RMSE'], test_metrics['R2'], test_metrics['MAPE']]
})
metrics_df.to_csv('model_metrics.csv', index=False)
print("✓ Model metrics saved: model_metrics.csv")


[STEP 7] Saving model and artifacts...
✓ Model saved: rf_model.pkl
✓ Feature names saved: input_features.pkl
✓ Feature importance saved: feature_importance.csv
✓ Model metrics saved: model_metrics.csv


In [11]:
# STEP 8: STORE REFERENCE STATISTICS
# ============================================================
print("\n[STEP 8] Creating store reference statistics...")

# Create aggregation dictionary
agg_dict = {}

if 'Store' in df.columns:
    # Add Size if exists
    if 'Size' in df.columns:
        agg_dict['Size'] = 'first'
    
    # Add Type_B if exists
    if 'Type_B' in df.columns:
        agg_dict['Type_B'] = 'first'
    
    # Add Type_C if exists
    if 'Type_C' in df.columns:
        agg_dict['Type_C'] = 'first'
    
    # Create store stats
    if agg_dict:
        store_stats = df.groupby('Store').agg(agg_dict).reset_index()
        
        # Rename columns to match expected format
        column_mapping = {
            'Size': 'size',
            'Type_B': 'type_b',
            'Type_C': 'type_c'
        }
        store_stats.columns = ['Store'] + [column_mapping.get(col, col.lower()) for col in store_stats.columns[1:]]
        
        # Add rolling statistics if available
        if 'Weekly_Sales_Rolling4' in df.columns:
            rolling_stats = df.groupby('Store')['Weekly_Sales_Rolling4'].median().reset_index()
            rolling_stats.columns = ['Store', 'rolling_median']
            store_stats = store_stats.merge(rolling_stats, on='Store', how='left')
            print("✓ Added rolling statistics")
        
        store_stats.to_csv('store_reference_stats.csv', index=False)
        print(f"✓ Store reference stats saved: store_reference_stats.csv ({len(store_stats)} stores)")
    else:
        print("⚠ Warning: No store characteristics found, skipping store stats")
else:
    print("⚠ Warning: No 'Store' column found, skipping store stats")


[STEP 8] Creating store reference statistics...
✓ Added rolling statistics
✓ Store reference stats saved: store_reference_stats.csv (2 stores)


In [12]:
# STEP 9: TEST PREDICTION (VERIFICATION)
# ============================================================
print("\n[STEP 9] Verification - Testing Prediction...")

# Get first row of test data
test_sample = X_test.iloc[0:1].copy()
actual_value = y_test.iloc[0]
predicted_value = rf_model.predict(test_sample)[0]

print(f"\nTest Sample Prediction:")
print(f"  Actual Sales:    ${actual_value:,.2f}")
print(f"  Predicted Sales: ${predicted_value:,.2f}")
print(f"  Difference:      ${abs(actual_value - predicted_value):,.2f}")
print(f"  Error %:         {abs(actual_value - predicted_value) / actual_value * 100:.2f}%")

# Show sample input values
print(f"\nSample Input Features:")
for col in test_sample.columns[:10]:  # Show first 10 features
    print(f"  {col}: {test_sample[col].values[0]}")
if len(test_sample.columns) > 10:
    print(f"  ... and {len(test_sample.columns) - 10} more features")

# ============================================================
# SUMMARY
# ============================================================
print("\n" + "="*60)
print("MODEL TRAINING COMPLETE!")
print("="*60)
print(f"\n✓ Model Type: Random Forest Regressor")
print(f"✓ Number of Trees: 300")
print(f"✓ Max Depth: 15")
print(f"✓ Features: {len(feature_names)}")
print(f"✓ Training Samples: {len(X_train)}")
print(f"✓ Test Samples: {len(X_test)}")
print(f"\n📊 Model Performance:")
print(f"  • Test R² Score: {test_metrics['R2']:.4f}")
print(f"  • Test MAE: ${test_metrics['MAE']:,.2f}")
print(f"  • Test RMSE: ${test_metrics['RMSE']:,.2f}")
print(f"  • Test MAPE: {test_metrics['MAPE']:.2f}%")
print(f"\n📁 Files Generated:")
print("  1. ✓ rf_model.pkl - Trained Random Forest model")
print("  2. ✓ input_features.pkl - Feature names list")
print("  3. ✓ feature_importance.csv - Feature importance rankings")
print("  4. ✓ model_metrics.csv - Model performance metrics")
print("  5. ✓ store_reference_stats.csv - Store-level statistics")
print("\n🚀 Ready for deployment!")
print("   Run: streamlit run app.py")
print("="*60)


[STEP 9] Verification - Testing Prediction...

Test Sample Prediction:
  Actual Sales:    $13,557.11
  Predicted Sales: $13,554.48
  Difference:      $2.63
  Error %:         0.02%

Sample Input Features:
  Store: 1
  Dept: 49
  Temperature: 91.65
  Fuel_Price: 3.684
  MarkDown1: 0.0
  MarkDown2: 0.0
  MarkDown3: 0.0
  MarkDown4: 0.0
  MarkDown5: 0.0
  CPI: 215.544618
  ... and 12 more features

MODEL TRAINING COMPLETE!

✓ Model Type: Random Forest Regressor
✓ Number of Trees: 300
✓ Max Depth: 15
✓ Features: 22
✓ Training Samples: 8000
✓ Test Samples: 2000

📊 Model Performance:
  • Test R² Score: 0.9998
  • Test MAE: $35.91
  • Test RMSE: $276.21
  • Test MAPE: inf%

📁 Files Generated:
  1. ✓ rf_model.pkl - Trained Random Forest model
  2. ✓ input_features.pkl - Feature names list
  3. ✓ feature_importance.csv - Feature importance rankings
  4. ✓ model_metrics.csv - Model performance metrics
  5. ✓ store_reference_stats.csv - Store-level statistics

🚀 Ready for deployment!
   Run: str